In [2]:
from torch_snippets import *

In [3]:
files = Glob("../data/drive_files/*.json")
files = sorted(files, key=lambda x: int(x.stem))
# files = files[:2]

In [5]:
x = {}
for f in track2(files):
    x.update(read_json(f))

x = AD(x)

In [6]:
def build_drive_tree(flat_files):
    # Map each ID to its node
    nodes = {}
    for file_id, meta in flat_files.items():
        node = {
            "id": meta.id,
            "name": meta.name,
            "mimeType": meta.mimeType,
            "modifiedTime": meta.modifiedTime,
        }
        if meta.mimeType == "application/vnd.google-apps.folder":
            node["children"] = {}
        nodes[file_id] = node

    # Build tree structure by linking parents to children
    tree = {}
    for file_id, meta in flat_files.items():
        node = nodes[file_id]
        parents = getattr(meta, "parents", [])
        if parents:
            parent_id = parents[0]  # assuming single-parent
            if parent_id in nodes:
                if meta.name == "d":
                    meta.name = "__d"
                nodes[parent_id].setdefault("children", {})[meta.name] = node
            else:
                # Orphaned file, missing parent
                if meta.name == "d":
                    meta.name = "__d"
                tree[meta.name] = node
        else:
            # Top-level (no parent)
            if meta.name == "d":
                meta.name = "__d"
            tree[meta.name] = node

    return AD(tree)


tree = build_drive_tree(x)
%env AD_MAX_ITEMS=10000000
tree.write(
    "../data/drive_tree.txt",
)

env: AD_MAX_ITEMS=10000000


In [7]:
write_json(tree, "../data/drive_tree.json")

» ../data/drive_tree.json